<a href="https://colab.research.google.com/github/akshay-r13/ds_from_scratch/blob/main/17_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello")